In [1]:
import pandas as pd
from pathlib import Path
import json
import os


def judging_to_csv(judge_type="val_judging"):
    judging_path = Path(os.getcwd()) / f"experiments/experiment_data/composition_experiment/{judge_type}"

    data = []

    for json_file in judging_path.glob("*.json"):
        with open(json_file, 'r') as file:
            content = json.load(file)
        
        header_info = content["HEADER"]
        row_data = header_info.copy()
        
        for key, value in content.items():
            if key != "HEADER":
                row_data[key] = value["asr"]
        data.append(row_data)

    df = pd.DataFrame(data)
    df.fillna(value=pd.NA, inplace=True)

    # Define the path for saving the DataFrame as a CSV
    csv_output_path = Path(os.getcwd()) / "experiments/experiment_data/composition_experiment/csvs"
    csv_output_path.mkdir(parents=True, exist_ok=True)
    csv_file_path = csv_output_path / f"{judge_type}.csv"
    df.to_csv(csv_file_path, index=False)

    return csv_file_path


csv_file_path = judging_to_csv()


In [2]:
df = pd.read_csv(csv_file_path)
# Extract the columns containing ASR values, which are all columns after the first four hyperparameter columns
asr_columns = df.columns[4:]

# Calculate the max, min, mean, std for each row for the ASR columns
df['max_asr'] = df[asr_columns].max(axis=1, skipna=True)
df['min_asr'] = df[asr_columns].min(axis=1, skipna=True)
df['mean_asr'] = df[asr_columns].mean(axis=1, skipna=True)
df['std_asr'] = df[asr_columns].std(axis=1, skipna=True)

# Reorder the columns to place the new stats columns right after the hyperparameter columns
hyperparameter_columns = df.columns[:4]
new_asr_stats_columns = ['max_asr', 'min_asr', 'mean_asr', 'std_asr']
df = df[hyperparameter_columns.tolist() + new_asr_stats_columns + asr_columns.tolist()]

df.to_csv(csv_file_path, index=False)



In [3]:
from experiments.composition import CompositionExperiment
from models.black_box_model import GPT4_Turbo
from string_transformations.string_transformations import Binary, Id, Leetspeak


hparams = {
    "k_num_transforms": 4,
    "maybe_transformation_instructions": True,
    "other_transform": Leetspeak.construct(),
    "composition_target": "response",
}
target_model = GPT4_Turbo()

ce = CompositionExperiment(target_model=target_model, num_attack_trials=0, hyperparameter_grid={k: [v,] for k, v in hparams.items()})

ce.data_path(**hparams)

/Users/brianhuang/miniconda3/envs/shieldbreak/lib/python3.12/site-packages/tqdm-4.66.2-py3.12.egg/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ltang (crtsn). Use `wandb login --relogin` to force relogin


'fb9ebbd7a739bf2576092085058c19c4ccb051ab127b9c75'

wandb: Network error (ConnectionError), entering retry loop.
